# Websocket Client Example
This client sends data to a web socket. Little does it know that the data then flows to a C application (over a regular socket from the WS server), then comes back here for processing. Nifty.

## Before you use this, read!
Before you try to use this guy, you have a few things to do first:

* Make sure the C application is running and listening.
* Make sure the server is running (do that after starting the C app). See the README.md for the repo. You'll need the right environment enabled if you're using one. And other things. Read it. to start the server: `python server_main.py` from the server_side directory.

Go forth! Send data. Just read the cell doc before doing things in the cell.

## Setup the Websocket Client
Make a client object and connect it to the WS server.

In [ ]:
from wsclient.client import TestWebSocketClient
from tornado import ioloop

print("IOLoop: %s" % (ioloop.IOLoop.current()))
client = TestWebSocketClient()
client.connect('ws://localhost:8675/ws')

## Visualization
Don't use this right now. Needs to be hooked up to the new message types and data structures

In [ ]:
%matplotlib notebook

# DEPRECATED. WON'T WORK RIGHT NOW (until the client is modded to accept the right new msg format)
# TODO: Make work with new message format!
import numpy as np
import matplotlib.pyplot as plt
import json
import random

class Plotter():
    """
    Wrapper class for the plt import with some helpers and a callback for the client.
    """
    bwidth = 0.5
    
    def __init__(self):
        """
        Initialize the object
        """
        # values sent mapped to the number of times sent
        self.val_counts = {}

        # width of the bars for the chart
        self.fig = None
        self.ax = None
        self.rects = []
        self.colors = {}
    
    def random_color(self):
        """
        Generate a random color (hex string).
        """
        lvls = range(32,256,32)
        t = tuple(random.choice(lvls) for _ in range(3))
        return "#%02x%02x%02x" % t

    def labelrects(self):
        """
        Helper for labelling the bars
        """
        # label the bars
        for rect in self.rects:
            height = rect.get_height()
            self.ax.text(rect.get_x() + rect.get_width()/2., 
                    0,
                    '%d' % int(height),
                    ha='center', 
                    va='bottom')


    def setup_plot(self):
        """
        Initialize the plot
        """
        self.fig, self.ax = plt.subplots()

        # add some text for y label and title
        self.ax.set_ylabel('Counts')
        self.ax.set_title('Counts of values sent to the WSServer')

        self.update_plot()

    
    def update_plot(self):
        """
        Update the plot with the current values of val_counts
        """
        plt.cla()
        vals = tuple(k for k,v in self.val_counts.items())
        counts = [v for k,v in self.val_counts.items()]
        
        for v in vals:
            self.colors.setdefault(v, self.random_color())
        
        # x locations for counts
        ind = np.arange(len(self.val_counts))

        self.rects = self.ax.bar(ind, counts, Plotter.bwidth, color=[c for v, c in self.colors.items()], align='center')

        # add some text for ticks
        self.ax.set_xticks(ind)
        self.ax.set_xticklabels(vals)

        # if we have some rects, set their heigthts, label
        if len(self.rects) > 0:
            for rect, h in zip(self.rects, counts):
                rect.set_height(h)
            self.ax.legend((self.rects), tuple('%s counts' % (v) for v in vals))
            self.labelrects()
        
        # redraw
        self.fig.canvas.draw()

    
    def client_msg_callback(self, msg):
        """
        Callback for the client to call when a message is received.
        """
        # TODO: Change msg to be a dict or other object before getting here...
        d = json.loads(msg.replace("'", "\""))
        if 'value' in d:
            # message contains a 'value' key. use it's value to update
            # the val_counts dict.
            self.val_counts[d['value']] = self.val_counts.setdefault(d['value'], 0) + 1
            self.update_plot()

p = Plotter()
# Better way to do this? callback feel wrong...
client.msg_callback(p.client_msg_callback)
p.setup_plot()

## Data Entry UI

In [ ]:
import json
from ipywidgets import (
    Box,
    HBox,
    Button,
    ToggleButtons,
    RadioButtons,
    IntSlider,
    Text,
    Select,
    HTML,
)
from IPython.display import display

# TODO: make this look better and refactor "make it work" code
# TODO: So much DRY violation...
# TODO: don't redefine things that are in the headers (like the val of the toggle buttons)
# TODO: do the events/interaction right, or at least better...look at interact

## SC Header:
### ToggleButtons type - (Get Request: 10 or Set Request: 100) - shouldn't be redefined here since in header
### Radio status (no real values defined, good/bad)
### IntSlider code (no real values defined)
### len comes from server side

header_label = HTML(
    value="<h2>Message Header Values</h2>",
)

htype_tb = ToggleButtons(
    description='Message Type:',
    options={'Get Request': 10, 'Set Request': 100,},
)

hstatus_rb = RadioButtons(
    description='Status:',
    options={'Good!': 1, 'Bad!': 2, 'Meh...': 3,},
)

hcode_is = IntSlider(description='Code', min=0, max=10)

header_container = Box(children=[header_label, htype_tb, hstatus_rb, hcode_is])
header_container.background_color = 'lightgray'
header_container.border_color = 'gray'
header_container.border_width = '3px'
header_container.border_radius = '3px'
header_container.padding = '10px'
header_container.margin = '10px'


## List Item (should be able to add up to 64, print em)
### IntSlider type - (no real values defined)
### scmsgtype is same as above, do at server
### Text name - 32 char max

litem_label = HTML(
    value="<h2>List Item Values</h2>",
)

litype_is = IntSlider(description='Type', min=0, max=10)
liname_txt = Text(
    description='Item Name:',
    value='Name me',
)
liadd_btn = Button(description='Add the list item!')


litem_vals = Box(children=[litem_label, litype_is, liname_txt,])
litem_vals.background_color = 'lightgray'

litem_list = HTML(value="<ul></ul>",)
litem_list.margin = '3px'
litem_list.padding = '3px'

items_container = Box(children=[litem_list])
items_container.background_color = 'lightgray'
items_container.border_color = 'gray'
items_container.border_width = '3px'
items_container.border_radius = '3px'
items_container.padding = '3px'
items_container.margin = '3px'


litem_container = Box(children=[litem_vals, liadd_btn, items_container,])
litem_container.background_color = 'lightgray'
litem_container.border_color = 'gray'
litem_container.border_width = '3px'
litem_container.border_radius = '3px'
litem_container.padding = '10px'
litem_container.margin = '10px'


send_btn = Button(description='Send message')
clear_btn = Button(description='Clear contents')

btn_container = HBox(children=[send_btn, clear_btn,])
btn_container.background_color = 'lightgray'
btn_container.border_color = 'gray'
btn_container.border_width = '3px'
btn_container.border_radius = '3px'
btn_container.padding = '10px'
btn_container.margin = '10px'

display(header_container)
display(litem_container)
display(btn_container)

# TODO: don't keep these global in this scope...
# TODO: this should be shareable with the server_side...
msg = {
    'header': {
        'type': None,
        'status': None,
        'code': None,
    },
    'listitems': [], # list of dicts
}

def add_list_item(b):
    """
    Add the item to the list of items, then update the 
    """
    ns = liname_txt.value if len(liname_txt.value) < 32 else liname_txt.value[0:32] 
    d = {
        'type': litype_is.value,
        'name': ns,
    }
    msg['listitems'].append(d)
    update_item_list()

def update_item_list():
    # TODO: do this smarter
    v = '<ul>'
    for i in msg['listitems']:
        v = v + '<li>Type: [%s] Name: [%s]</li>' % (i['type'], i['name'])
    v = v + '</ul>'
    litem_list.value = v

def clear_all(b):
    # TODO: like all the other ugly stuff, do this right...
    msg['header']['type'] = None
    msg['header']['status'] = None
    msg['header']['code'] = None
    msg['listitems'] = []    
    update_item_list()

def send_msg(b):
    msg['header']['type'] = htype_tb.value
    msg['header']['status'] = hstatus_rb.value
    msg['header']['code'] = hcode_is.value
    # list items should be set already...
    print('send_msg: %s' % (msg))
    client.send(msg)

def client_msg_callback(msg):
        """
        Callback for the client to call when a message is received.
        """
        # TODO: figure out what needs to happen here...
        # TODO: Change msg to be a dict or other object before getting here...
        d = json.loads(msg.replace("'", "\""))
        print('W00T! Got a message from the WS server: %s' % (d))
            
liadd_btn.on_click(add_list_item)
send_btn.on_click(send_msg)
clear_btn.on_click(clear_all)

# Better way to do this? callback feel wrong...
client.msg_callback(client_msg_callback)


Connection to Websocket Server closed!


## Remove when associated code is changed above

In [ ]:
# DON'T USE
#from ipywidgets import Button, IntSlider
#from IPython.display import display

# TODO: deprecate!


#button = Button(description='Send a test Message!')
#valinput = IntSlider(description='Enter a number to send', min=0, max=10)

#display(valinput)
#display(button)

#def send_test_msg(b):
#    d = {
#        'does_it': 'blend',
#        'value': valinput.value,
#    }
#    print('send_test_msg: sending test %s' % (d))
#    client.send(d)

#button.on_click(send_test_msg)